In [ ]:
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

def similarity(a: str, b: str) -> float:
    emb_1 = encoder.encode(a)
    emb_2 = encoder.encode(b)

    return round(float(encoder.similarity(emb_1, emb_2).item()), 3)

dataset = load_dataset('IlyaGusev/librusec_full', split='train', streaming=True)

with open('All_annotations.json', 'r', encoding='utf-8') as f:
    all_a = json.load(f)

with open('titles.json', 'r', encoding='utf-8') as f:
    needed_titles = json.load(f)
    
annotations_dict = {}
for a in all_a:
    annotations_dict[a['title']] = a
     
SIM_TH = 0.65
texts_dict = {}
needed_titles = set(needed_titles)
encoder = SentenceTransformer('deepvk/USER-bge-m3')

with tqdm(total=496858, desc="Processing records") as pbar:
    for record in dataset:
        title = record.get("title", "")
        authors = record.get("authors", [""])
        lang = record.get("lang", "")

        if title in needed_titles:
            if title not in texts_dict:
                if lang in ['ru', 'rus']:
                    if annotations_dict[title]['author'] in authors:
                        texts_dict[title] = (record.get("sections", ""), authors)
                    else:
                        for author in authors:
                            if similarity(author, annotations_dict[title]['author']) > SIM_TH:
                                texts_dict[title] = (record.get("sections", " "), authors)
                                break

        pbar.set_postfix({
            'found': len(texts_dict),
        })
        pbar.update(1)

        if len(texts_dict) == len(needed_titles):
            break
            
combined = []
for title, value in texts_dict.items():
  combined.append({'title': title, 'author': annotations_dict[title]['author'], 'authors': value[1], 'annotation': annotations_dict[title]['annotation'],'text': value[0], 'categories': annotations_dict[title]['categories']})

Processing records: 100%|██████████| 496858/496858 [4:05:20<00:00, 33.75it/s, found=634]


In [5]:
import sys
sys.path.append('methods')

from methods import Summarisation

C:\Users\dagri\BOOOKSum\utils.py:4: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.special import softmax
C:\Users\dagri\anaconda3\envs\WIKIbench\Lib\site-packages\ignite\handlers\checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [6]:
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

In [7]:
bench = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta') 

In [8]:
lst = list(await bench.client.client.models.list())[0][1]
for l in lst:
    print(l)

Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-32B-Pro-Beta', created=None, object=None, owned_by=None, max_model_len=28000, status='spawned')
Model(id='RefalMachine/RuadaptQwen3-32B-Instruct-v2', created=None, object=None, owned_by=None, max_model_len=24000, status='spawned')
Model(id='llama3-70b', created=None, object=None, owned_by=None, max_model_len=14000, status='spawned')
Model(id='DeepSeek V3', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen/Qwen2.5-VL-72B-Instruct', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen3-235B-A22B', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Garant-v2', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='deepseek-r1-32b', cr

In [7]:
from utils import chunk_text
text = "\n".join(bench.collection[3]['text'])
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

870892
109


In [4]:
from utils import chunk_text
text = bench.collection[0]['text'][0]
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

60664
8


In [8]:
s1 = await bench.cluster_blueprint.run(chunks, initial_word_limit=500)

In [9]:
print(s1)

### Краткое содержание:

**Эмоциональные реакции Мейзи:**
- Мейзи испытывает смесь восторга от красоты и магии Волвердена, особенно восхищаясь елизаветинской архитектурой, но испытывает глубокий страх и беспокойство перед башней и старушкой Бесси, что отражает её недоверие к мистическим аспектам и предчувствие надвигающейся опасности.

**Характеристики Волвердена:**
- Замок, совмещающий готические и елизаветинские элементы, символизирует защиту и магическую силу, защищённый трёхслойной магией и древними символами, что подчеркивает его мистическую значимость и древнее наследие.

**Значение старухи Бесси:**
- Старуха Бесси, хранительница древних знаний, играет ключевую роль в местной мифологии, её присутствие в башне Волвердена усиливает мистическую атмосферу, вызывая одновременно страх и глубокое уважение среди обитателей, будучи связующей нитью с прошлым Братства.

**Развитие событий и переживаний Мейзи:**
- Путь Мейзи от первоначального восторга к глубокому посвящению в Братство включ

In [6]:
print(s)

В начале XX века, в живописных окрестностях кентского Вильде, где каждый камень хранит древние тайны, разворачивается история, связанная с таинственным замком Волверден, недавно обретшим второе дыхание благодаря стараниям его нового хозяина – полковника Уэст. Этот величественный особняк, где архитектура эпохи Елизаветы изящно переплетается с изысканными элементами, становится не просто жилищем, а живым зеркалом столетий, окутанным легендами и мистикой.

В центр повествования попадает Мейзи Льюэллин, дочь антиквара, чья глубокая связь с прошлым и страстью к антиквариату словно магнит притягивает её к Волвердену. Приезд Мейзи в замок, организованный заботливой миссис Уэст, хозяйкой, преданной традициям и хранительницей старинного очага, знаменует начало её погружения в атмосферу, насыщенную предчувствием и мистикой.

С первых шагов Мейзи увлекает реставрация замка, особенно таинственная башня, символизирующая не столько физическую реконструкцию, сколько попытку сохранить хрупкие нити про

In [7]:
await bench.evaluater.evaluate_annotation(bench.collection[0]['annotation'], s)

['Кто является главной героиней истории и куда она приглашена перед Рождеством?', 'Какова история башни Волверден и почему ее пришлось реконструировать?', 'Кто такая старуха Бесси и что она рассказывает Мэйзи о башне Волверден?', 'Кто такие две загадочные девушки, которых Мэйзи встречает на ужине, и что они представляют собой?', 'Что происходит с Мэйзи, когда она идет на прогулку с подругами и попадает в склеп?', 'Какова цель ритуала, в котором Мэйзи становится участницей, и что ожидается от нее?', 'Кто спасает Мэйзи от участи стать третьей жертвой и что происходит с башней Волверден в конце истории?']


['Двадцатилетняя дочь антиквара Мэйзи Льюэллин приглашена погостить в замок елизаветинской эпохи Волверден-холл в окрестностях Лондона перед Рождеством.', 'Башня Волверден была старинной башней, которая, согласно легенде, была трижды скреплена и укреплена душами девушек, чтобы противостоять натиску людей и дьявола. Однако со временем башня пришла в плачевное состояние и ее пришлось рек

((np.float32(0.64318556), np.float32(0.54767674), np.float32(0.59160113)),
 0.4393,
 0.2857142857142857,
 0.2365714285714286)

In [10]:
await bench.evaluater.evaluate_annotation(bench.collection[0]['annotation'], s1)

['Кто является главной героиней истории и куда она приглашена перед Рождеством?', 'Какова история старинной башни Волверден и почему её пришлось реконструировать?', 'Кто такая старуха Бесси и что она рассказывает Мэйзи о башне Волверден?', 'Что происходит с Мэйзи во время ужина и живых картин в замке Волверден-холл?', 'Кто такие Иоланта и Гедда, и как они появляются в жизни Мэйзи?', 'Что происходит с Мэйзи, когда она идет на прогулку с Иолантой и Геддой, и куда они ее ведут?', 'Что такое Братство Мёртвых Зодчих и Жертв пределов Волвердена, и какова их роль в истории?', 'Почему Мэйзи соглашается стать третьей жертвой, и что происходит с ней на верхней площадке башни?', 'Кто спасает Мэйзи от жертвы, и что происходит с ней после этого?', 'Что происходит с башней Волверден в конце истории, и как Мэйзи реагирует на это событие?']


['Двадцатилетняя дочь антиквара Мэйзи Льюэллин приглашена погостить в замок елизаветинской эпохи Волверден-холл в окрестностях Лондона перед Рождеством.', 'Стари

((np.float32(0.6147925), np.float32(0.51682734), np.float32(0.56156945)),
 0.3215,
 0.1,
 0.06949999999999999)

In [8]:
import time
print('Hierarchical')
for _ in range(3):
    start = time.perf_counter()
    s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
    end = time.perf_counter()
    count = end - start
    print(f"Time: {count:.2f}")
    break

Hierarchical
Time: 166.78


In [9]:
import time
print('Blueprint')
for _ in range(3):
    start = time.perf_counter()
    s = await bench.blueprint.run(chunks, initial_word_limit=500)
    end = time.perf_counter()
    count = end - start
    print(f"Time: {count:.2f}")

Blueprint
Time: 533.10
Time: 522.88
Time: 537.34


In [5]:
from utils import chunk_text

for item in bench.collection[:2]:
    text = '\n'.join(item['text'])
    chunks = chunk_text(text)
    s = await bench.blueprint.run(chunks, initial_word_limit=500)
    print('Blueprint')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.cluster_blueprint.run(chunks, initial_word_limit=500)
    print('Cluster blueprint')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.hierarchical.run(chunks, initial_word_limit=500)
    print('Hierarchical')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
    print('Hierarchical filtered')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))

Blueprint
((np.float32(0.65899706), np.float32(0.5885342), np.float32(0.62177575)), 0.4988, 0.5714285714285714, 0.454)
Cluster blueprint
((np.float32(0.62056816), np.float32(0.53078747), np.float32(0.57217735)), 0.3997, 0.1111111111111111, 0.081)
Hierarchical
((np.float32(0.6443127), np.float32(0.55432045), np.float32(0.59593827)), 0.4336, 0.5714285714285714, 0.43842857142857145)
Hierarchical filtered
((np.float32(0.6438272), np.float32(0.55141735), np.float32(0.59404993)), 0.4317, 0.5714285714285714, 0.441)
Blueprint
((np.float32(0.57740545), np.float32(0.5320905), np.float32(0.5538226)), 0.2276, 0.2727272727272727, 0.1969090909090909)
Cluster blueprint
((np.float32(0.54771286), np.float32(0.49291065), np.float32(0.51886874)), 0.3262, 0.18181818181818182, 0.12854545454545455)
Hierarchical
((np.float32(0.56958234), np.float32(0.4970916), np.float32(0.5308737)), 0.3647, 0.1, 0.09290000000000001)
Hierarchical filtered
((np.float32(0.5903005), np.float32(0.51581717), np.float32(0.55055106

In [6]:
for item in bench.collection[:2]:
    text = '\n'.join(item['text'])
    chunks = chunk_text(text)
    s = await bench.blueprint.run(chunks, initial_word_limit=500)
    print('Blueprint')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.cluster_blueprint.run(chunks, initial_word_limit=500)
    print('Cluster blueprint')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.hierarchical.run(chunks, initial_word_limit=500)
    print('Hierarchical')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))
    s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
    print('Hierarchical filtered')
    print(await bench.evaluater.evaluate_annotation(item['annotation'], s))

Blueprint
((np.float32(0.61450547), np.float32(0.5522304), np.float32(0.581706)), 0.3177, 0.42857142857142855, 0.3605714285714286)
Cluster blueprint
((np.float32(0.544789), np.float32(0.53977144), np.float32(0.54226863)), 0.3211, 0.4, 0.3009)
Hierarchical
((np.float32(0.63047796), np.float32(0.55124193), np.float32(0.5882035)), 0.4056, 0.5714285714285714, 0.42742857142857144)
Hierarchical filtered
((np.float32(0.64241886), np.float32(0.5443015), np.float32(0.58930403)), 0.3741, 0.5, 0.3815)
Blueprint
((np.float32(0.5616741), np.float32(0.51038224), np.float32(0.5348012)), 0.3222, 0.2222222222222222, 0.15288888888888888)
Cluster blueprint
((np.float32(0.55782133), np.float32(0.51458544), np.float32(0.5353318)), 0.4035, 0.3, 0.2575)
Hierarchical
((np.float32(0.5558575), np.float32(0.5063232), np.float32(0.5299353)), 0.3839, 0.2, 0.1477)
Hierarchical filtered
((np.float32(0.58505607), np.float32(0.51965725), np.float32(0.5504209)), 0.449, 0.36363636363636365, 0.2928181818181818)


In [ ]:
from utils import chunk_text
from tqdm import tqdm


results_b = []
results_bc = []
results_h = []
results_hf = []

count = 0

for item in tqdm(bench.collection[:100], total=100):
    #print(count)
    text = '\n'.join(item['text'])
    chunks = chunk_text(text)
    s = await bench.blueprint.run(chunks, initial_word_limit=500)
    #print('Blueprint')
    r = await bench.evaluater.evaluate_annotation(item['annotation'], s)
    #print(r)
    results_b.append(s)
    s = await bench.cluster_blueprint.run(chunks, initial_word_limit=500)
    #print('Cluster blueprint')
    r = await bench.evaluater.evaluate_annotation(item['annotation'], s)
    #print(r)
    results_bc.append(s)
    s = await bench.hierarchical.run(chunks, initial_word_limit=500)
    #print('Hierarchical')
    r = await bench.evaluater.evaluate_annotation(item['annotation'], s)
    #print(r)
    results_h.append(s)
    s = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
    #print('Hierarchical filtered')
    r = await bench.evaluater.evaluate_annotation(item['annotation'], s)
    #print(r)
    results_hf.append(s)
    #count += 1

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
b_q = [e[3] for e in results_b]

In [11]:
print(max(b_q))

0.748


In [12]:
i = 0
for _ in range(len(b_q)):
    if b_q[i] != 0.748:
        i += 1
    else:
        break
print(i)

36


In [15]:
bench.collection[36]['annotation']

'Идёт суд над босым и худым мужичком Денисом Григорьевым. Его обвиняют в отвинчивании гайки, которой рельсы крепятся к шпалам. Мужичок этого не отрицает, но своей вины не видит. Следователь выясняет, что Денис, как и другие климовские мужики, откручивает гайки для того, чтобы делать из них грузила. Подсудимый искренне не понимает, что такое отвинчивание может привести к аварии поезда и к гибели людей. Следователь отправляет злоумышленника в тюрьму, но тот по прежнему не понимает, что он такого сделал.'